In [20]:
import os
os.chdir("/Users/kunalsingh/Desktop/DL Project")
print("Current working directory:", os.getcwd())

Current working directory: /Users/kunalsingh/Desktop/DL Project


In [21]:
%pwd

'/Users/kunalsingh/Desktop/DL Project'

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [23]:
from  cnnClassifier import logger
from cnnClassifier.constants import (CONFIG_FILE_PATH,PARAMS_FILE_PATH)
from cnnClassifier.utils.common import read_yaml, create_directories


In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [28]:
import os
import urllib.request as request
import zipfile
from cnnClassifier.utils.common import get_size
import kagglehub
import shutil

In [29]:
class DataIngestion:
    def __init__(self,config : DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):

            # Download latest version
            path = kagglehub.dataset_download(self.config.source_URL)
            logger.info(f"Downloaded file from kaggle hub: {path}")
            # Move the downloaded file to self.config.root_dir
            dest_path = os.path.join(self.config.root_dir, "brain-mri")
            os.makedirs(self.config.root_dir, exist_ok=True)
            shutil.move(path, dest_path)
            logger.info(f"Moved file to: {dest_path}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file}")


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [30]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
except Exception as e:
    raise e

[2025-09-01 11:41:43,120 : INFO : common : yaml file: config/config.yaml loaded successfully]
[2025-09-01 11:41:43,122 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-09-01 11:41:43,123 : INFO : common : created directory at: artifacts]
[2025-09-01 11:41:43,124 : INFO : common : created directory at: artifacts/data_ingestion]
[2025-09-01 11:41:43,694 : INFO : 4025829298 : Downloaded file from kaggle hub: /Users/kunalsingh/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1]
[2025-09-01 11:41:43,780 : INFO : 4025829298 : Moved file to: artifacts/data_ingestion/brain-mri]
